# AirNow and CAM-chem

First we need to import the driver.

In [1]:
import sys
sys.path.append('../../')

In [2]:
from melodies_monet import driver

In [3]:
# Needed if you want to make changes to `melodies_monet` and don't want to restart kernel:
%load_ext autoreload

%autoreload 2

## Initiate the analysis class

Now lets create an instance of the {mod}`melodies_monet.driver` {class}`~melodies_monet.driver.analysis` class.
It consists of 4 main parts: model instances, observation instances, a paired instance of both.
This will allow us to move things around the plotting function for spatial and overlays and more complex plots.

In [4]:
an = driver.analysis()
an

analysis(
    control='control.yaml',
    control_dict=None,
    models={},
    obs={},
    paired={},
    start_time=None,
    end_time=None,
    download_maps=True,
    output_dir=None,
    debug=False,
)

## Control File

Read in the required yaml control file that sets up all the definitons of what we want to pair and plot.

In [5]:
an.control = 'control_camchem.yaml'
an.read_control()
an.control_dict

{'analysis': {'start_time': '2019-09-01-00:00:00',
  'end_time': '2019-09-09-00:00:00',
  'output_dir': './output/camchem',
  'download_maps': False,
  'debug': False},
 'model': {'cam-chem': {'files': 'example:camchem:fv',
   'mod_type': 'cesm_fv',
   'radius_of_influence': 18000,
   'mapping': {'airnow': {'O3': 'OZONE'}},
   'projection': 'None',
   'plot_kwargs': {'color': 'dodgerblue', 'marker': '+', 'linestyle': '-.'}}},
 'obs': {'airnow': {'use_airnow': True,
   'filename': 'example:airnow:2019-09',
   'obs_type': 'pt_sfc',
   'variables': {'PM2.5': {'unit_scale': 1,
     'unit_scale_method': '*',
     'nan_value': -1.0,
     'ylabel_plot': 'PM2.5 (ug/m3)',
     'ty_scale': 2.0,
     'vmin_plot': 0.0,
     'vmax_plot': 22.0,
     'vdiff_plot': 15.0,
     'nlevels_plot': 23},
    'OZONE': {'unit_scale': 1,
     'unit_scale_method': '*',
     'nan_value': -1.0,
     'ylabel_plot': 'Ozone (ppbv)',
     'vmin_plot': 15.0,
     'vmax_plot': 55.0,
     'vdiff_plot': 20.0,
     'nlevels

## Load the model data 

The driver will automatically loop through the "models" found in the model section of the control file and create model classes for each. Classes include the label, mapping information, and xarray object as well as the filenames.  Note it can open multiple files easily by including wildcards. Here we are only opening one CAM-chem file.

In [6]:
an.open_models()

cam-chem
{'files': 'example:camchem:fv', 'mod_type': 'cesm_fv', 'radius_of_influence': 18000, 'mapping': {'airnow': {'O3': 'OZONE'}}, 'projection': 'None', 'plot_kwargs': {'color': 'dodgerblue', 'marker': '+', 'linestyle': '-.'}}
example:camchem:fv
**** Reading CESM model output...


In [7]:
an.models

{'cam-chem': model(
     model='cesm_fv',
     radius_of_influence=18000,
     mod_kwargs={},
     file_str='example:camchem:fv',
     label='cam-chem',
     obj=...,
     mapping={'airnow': {'O3': 'OZONE'}},
     label='cam-chem',
     ...
 )}

In [8]:
an.models['cam-chem'].obj

<xarray.Dataset>
Dimensions:  (time: 36, z: 1, lat: 192, lon: 288)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * z        (z) float64 992.5
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 2019-09-01T06:00:00 ... 2019-09-10
Data variables:
    O3       (time, z, lat, lon) float32 dask.array<chunksize=(36, 1, 192, 288), meta=np.ndarray>
    PM25     (time, z, lat, lon) float32 dask.array<chunksize=(36, 1, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              fmerra.2.1003.FCSD.f09.qfedcmip.56L.001.branch02
    logname:           buchholz
    host:              cheyenne3
    initial_file:      /glade/p/cesmdata/cseg/inputdata/atm/cam/inic/fv/f.e20...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/met/MERRA2/0....
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  hour_6
    history:           Mon Feb 28 16:25:23 2022: ncks -7 -L 1 --baa=4 --ppc d...
    NCO:               netCDF Operators version 5.0.6 (Homepage = http://nco....

In [9]:
# All the info in the model class can be called here.
print(an.models['cam-chem'].label)
print(an.models['cam-chem'].mapping)

cam-chem
{'airnow': {'O3': 'OZONE'}}


In [10]:
# All the info in the analysis class can also be called.
print(an.start_time)
print(an.end_time)
print(an.download_maps)

2019-09-01 00:00:00
2019-09-09 00:00:00
True


## Open Obs

Now for monet-analysis we will open preprocessed data in either netcdf icartt or some other format.  We will not be retrieving data like monetio does for some observations (ie aeronet, airnow, etc....).  Instead we will provide utitilies to do this so that users can add more data easily.

Like models we list all obs objects in the yaml file and it will loop through and create driver.observation instances that include the model type, file, objects (i.e. data object) and label  

In [11]:
an.control_dict['obs']

{'airnow': {'use_airnow': True,
  'filename': 'example:airnow:2019-09',
  'obs_type': 'pt_sfc',
  'variables': {'PM2.5': {'unit_scale': 1,
    'unit_scale_method': '*',
    'nan_value': -1.0,
    'ylabel_plot': 'PM2.5 (ug/m3)',
    'ty_scale': 2.0,
    'vmin_plot': 0.0,
    'vmax_plot': 22.0,
    'vdiff_plot': 15.0,
    'nlevels_plot': 23},
   'OZONE': {'unit_scale': 1,
    'unit_scale_method': '*',
    'nan_value': -1.0,
    'ylabel_plot': 'Ozone (ppbv)',
    'vmin_plot': 15.0,
    'vmax_plot': 55.0,
    'vdiff_plot': 20.0,
    'nlevels_plot': 21}}}}

In [12]:
an.open_obs()

In [13]:
# All the info in the observation class can also be called.
an.obs['airnow'].obj

<xarray.Dataset>
Dimensions:     (x: 3786, time: 2091, y: 1)
Coordinates:
  * x           (x) int64 0 1 2 3 4 5 6 7 ... 3779 3780 3781 3782 3783 3784 3785
  * time        (time) datetime64[ns] 2019-09-01 ... 2019-09-30T00:30:00
    latitude    (y, x) float64 ...
    longitude   (y, x) float64 ...
    siteid      (y, x) object ...
Dimensions without coordinates: y
Data variables: (12/30)
    BARPR       (time, y, x) float64 ...
    BC          (time, y, x) float64 ...
    CO          (time, y, x) float64 ...
    NO          (time, y, x) float64 ...
    NO2         (time, y, x) float64 ...
    NO2Y        (time, y, x) float64 ...
    ...          ...
    cmsa_name   (y, x) float64 ...
    msa_code    (y, x) float64 ...
    msa_name    (y, x) object ...
    state_name  (y, x) object ...
    epa_region  (y, x) object ...
    time_local  (time, y, x) datetime64[ns] ...
Attributes:
    title:         
    format:        NetCDF-4
    date_created:  2021-06-07

## Pair model and obs data

In [14]:
an.pair_data()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.4s


## Generate plots

In [15]:
an.plotting()

Reference std: 15.44077321453672
Reference std: 9.392777428526704
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.4s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s


**10 Figures**

::::{card-carousel} 10

:::{card} Figure 1
:img-background: output/camchem/plot_grp1.timeseries.OZONE.2019-09-01_00.2019-09-09_00.all.CONUS.png
:width: 50%
:::

:::{card} Figure 2
:img-background: output/camchem/plot_grp1.timeseries.OZONE.2019-09-01_00.2019-09-09_00.epa_region.R1.png
:width: 50%
:::

:::{card} Figure 3
:img-background: output/camchem/plot_grp2.taylor.OZONE.2019-09-01_00.2019-09-09_00.all.CONUS.png
:width: 50%
:::

:::{card} Figure 4
:img-background: output/camchem/plot_grp2.taylor.OZONE.2019-09-01_00.2019-09-09_00.epa_region.R1.png
:width: 50%
:::

:::{card} Figure 5
:img-background: output/camchem/plot_grp3.spatial_bias.OZONE.2019-09-01_00.2019-09-09_00.all.CONUS.airnow_cam-chem.png
:width: 50%
:::

:::{card} Figure 6
:img-background: output/camchem/plot_grp3.spatial_bias.OZONE.2019-09-01_00.2019-09-09_00.epa_region.R1.airnow_cam-chem.png
:width: 50%
:::

:::{card} Figure 7
:img-background: output/camchem/plot_grp4.spatial_overlay.OZONE.2019-09-01_00.2019-09-09_00.all.CONUS.airnow_cam-chem.png
:width: 50%
:::

:::{card} Figure 8
:img-background: output/camchem/plot_grp4.spatial_overlay.OZONE.2019-09-01_00.2019-09-09_00.epa_region.R1.airnow_cam-chem.png
:width: 50%
:::

:::{card} Figure 9
:img-background: output/camchem/plot_grp5.boxplot.OZONE.2019-09-01_00.2019-09-09_00.all.CONUS.png
:width: 50%
:::

:::{card} Figure 10
:img-background: output/camchem/plot_grp5.boxplot.OZONE.2019-09-01_00.2019-09-09_00.epa_region.R1.png
:width: 50%
:::

::::